# Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Second Step: Data Preprocessing

In [3]:
df=pd.read_csv("Churn_Modelling.csv")

In [4]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [152]:
x=df.drop(["CustomerId","RowNumber","Surname","Exited"],axis=1)
y=df["Exited"]

### Categorical data to onehotencoder
#### What is different between One Hot Encoding and Label Encoding

# Label Encoding
Label Encoding is a popular encoding technique for handling categorical variables. In this technique, each label is assigned a unique integer based on alphabetical ordering.
Challenges with Label Encoding
In the above scenario, the Country names do not have an order or rank. But, when label encoding is performed, the country names are ranked based on the alphabets. Due to this, there is a very high probability that the model captures the relationship between countries such as India < Japan < the US.

# One Hot Encoding
One-Hot Encoding is another popular technique for treating categorical variables. It simply creates additional features based on the number of unique values in the categorical feature. Every unique value in the category will be added as a feature.
One-Hot Encoding is the process of creating dummy variables.

## Challenges of One-Hot Encoding: Dummy Variable Trap
One-Hot Encoding results in a Dummy Variable Trap as the outcome of one variable can easily be predicted with the help of the remaining variables.

Dummy Variable Trap is a scenario in which variables are highly correlated to each other.
The Dummy Variable Trap leads to the problem known as multicollinearity. Multicollinearity occurs where there is a dependency between the independent features. Multicollinearity is a serious issue in machine learning models like Linear Regression and Logistic Regression.

So, in order to overcome the problem of multicollinearity, one of the dummy variables has to be dropped

### Multicollinearity
One of the common ways to check for multicollinearity is the Variance Inflation Factor (VIF):

VIF=1, Very Less Multicollinearity
VIF<5, Moderate Multicollinearity
VIF>5, Extreme Multicollinearity (This is what we have to avoid)

# When to use a Label Encoding vs. One Hot Encoding
This question generally depends on your dataset and the model which you wish to apply. But still, a few points to note before choosing the right encoding technique for your model:

### We apply One-Hot Encoding when:

`1` The categorical feature is not ordinal (like the countries above)

`2` The number of categorical features is less so one-hot encoding can be effectively applied
### We apply Label Encoding when:

`1` The categorical feature is ordinal (like Jr. kg, Sr. kg, Primary school, high school)

`2` The number of categories is quite large as one-hot encoding can lead to high memory consumption

In [167]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
cat_feat=["Geography","Gender"]
one_hot=OneHotEncoder()
transformer=ColumnTransformer([("one_hot",one_hot,cat_feat)],
                              remainder="passthrough")
transformed_x=transformer.fit_transform(x)
df_transformed_x=pd.DataFrame(transformed_x)


In [170]:
df_transformed_x

,1,2,4,5,6,7,8,9,10,11,12
0,0.0,0.0,0.0,619.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88
1,0.0,1.0,0.0,608.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58
2,0.0,0.0,0.0,502.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57
3,0.0,0.0,0.0,699.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63
4,0.0,1.0,0.0,850.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,1.0,771.0,39.0,5.0,0.00,2.0,1.0,0.0,96270.64
9996,0.0,0.0,1.0,516.0,35.0,10.0,57369.61,1.0,1.0,1.0,101699.77
9997,0.0,0.0,0.0,709.0,36.0,7.0,0.00,1.0,0.0,1.0,42085.58
9998,1.0,0.0,1.0,772.0,42.0,3.0,75075.31,2.0,1.0,0.0,92888.52


### For Preventing Multicollinearity delete 0 and 4

In [169]:
df_transformed_x.drop([0,3],axis=1,inplace=True)

### Split Our Data to Train and Test

In [171]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_transformed_x,y,
                                               test_size=.2,
                                               random_state=0)

## because of large computing in ANN we need to Feature Scaling

In [174]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)

In [175]:
x_train

array([[-0.5698444 ,  1.74309049, -1.09168714, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368,  0.91601335, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.09168714, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 , -0.57369368,  0.91601335, ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -1.09168714, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -1.09168714, ...,  0.64259497,
        -1.03227043,  0.32472465]])

In [177]:
x_test=sc.transform(x_test)

# Building ANN Model

In [178]:
import keras

In [179]:
from keras.models import Sequential

In [180]:
from keras.layers import Dense

## Initializing The ANN
as you remeber that wights will going to be near zero but not zero

In [181]:
classifier=Sequential()

### add layers
we're goning too add input layer and first hidden layer

`Dense Function`:

`output_dim`,`Rule of thumbs`  number of nodes in hidden layer (output_dim):
you can average between number of nodes in input layer and number of nodes in output layer but in reality or for getting better outcome yoy should use k-fold cross validation and tune your parameter.`note that the output_dim is changed to unit and you don't need to specify output_dim and just use the number`

`kernel_initializer`:  value of weights for initializing should be uniform.

`activation`:  activation function that we want to use, here for hidden layer we want to use rectifier actication function and fot output layer we want to use sigmoidactivation function ('relu':for rectifier)

`input_dim`: because we just initialize our ANN and it does'nt know how many idependent var we have, and we don't specify input and output layer we should use this parameter for first hidden layer.
## for more information about keras.Dense
https://keras.io/api/layers/core_layers/dense/

In [184]:
classifier.add(Dense(6,kernel_initializer="uniform",activation="relu",input_dim=11))

### Add Second Hidden Layer

As the Second Hidden layer get data from first hidden layer the we don't need to specify input_dim any more

In [185]:
classifier.add(Dense(6,kernel_initializer="uniform",activation="relu"))

## Add Output Layer

`"sigmoid"for sigmoid activation function` and if you dealing with dependent var that have more than two categories first the output_dim set to number of classes and second activation function is "softmax" (it is indeed sigmoid function)

In [186]:
classifier.add(Dense(1,kernel_initializer="uniform",activation="sigmoid"))

## Compile the ANN

`optimizer`:  the optimize method that we want to use to find out the best wights and "adam" is one of stochastic  gradient descent algorithm

`loss`: loss function as the error will show here and we need to optimize it.
if you have two categorical out come: loss function will be binary_crossentropy and if you have more than two categorical_crossentropy. note that this is only for sigmoid activation function and these losses are logaritmic. 

`metrics`: way that you choose to evaluate your model

In [187]:
classifier.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

## Train Our Model

`Step 6 and Step 7`: Reinforment vs Batch Learning , number of epoch



In [189]:
classifier.fit(x_train,y_train,batch_size=10,epochs=100)

Epoch 1/100
800/800 [==============================] - 1s 971us/step - loss: 0.4796 - accuracy: 0.7960
Epoch 2/100
800/800 [==============================] - 1s 949us/step - loss: 0.4283 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 915us/step - loss: 0.4207 - accuracy: 0.8094
Epoch 4/100
800/800 [==============================] - 1s 873us/step - loss: 0.4147 - accuracy: 0.8276
Epoch 5/100
800/800 [==============================] - 1s 909us/step - loss: 0.4100 - accuracy: 0.8322
Epoch 6/100
800/800 [==============================] - 1s 920us/step - loss: 0.4075 - accuracy: 0.8345
Epoch 7/100
800/800 [==============================] - 1s 879us/step - loss: 0.4045 - accuracy: 0.8344
Epoch 8/100
800/800 [==============================] - 1s 861us/step - loss: 0.4032 - accuracy: 0.8331
Epoch 9/100
800/800 [==============================] - 1s 869us/step - loss: 0.4025 - accuracy: 0.8347
Epoch 10/100
800/800 [==============================] - 1s 865us/step - l

800/800 [==============================] - 1s 891us/step - loss: 0.3492 - accuracy: 0.8597
Epoch 80/100
800/800 [==============================] - 1s 860us/step - loss: 0.3488 - accuracy: 0.8591
Epoch 81/100
800/800 [==============================] - 1s 870us/step - loss: 0.3488 - accuracy: 0.8608
Epoch 82/100
800/800 [==============================] - 1s 909us/step - loss: 0.3483 - accuracy: 0.8602
Epoch 83/100
800/800 [==============================] - 1s 935us/step - loss: 0.3474 - accuracy: 0.8600
Epoch 84/100
800/800 [==============================] - 1s 905us/step - loss: 0.3474 - accuracy: 0.8608
Epoch 85/100
800/800 [==============================] - ETA: 0s - loss: 0.3475 - accuracy: 0.86 - 1s 861us/step - loss: 0.3476 - accuracy: 0.8608
Epoch 86/100
800/800 [==============================] - 1s 877us/step - loss: 0.3468 - accuracy: 0.8604
Epoch 87/100
800/800 [==============================] - 1s 866us/step - loss: 0.3472 - accuracy: 0.8596
Epoch 88/100
800/800 [=============

## Predict the test set

In [190]:
y_pred=classifier.predict(x_test)

In [192]:
y_pred

array([[0.23362735],
       [0.3744532 ],
       [0.1683062 ],
       ...,
       [0.22024369],
       [0.15126762],
       [0.19625965]], dtype=float32)

## Evaluate with Confusion Matrix

First Convert the predicted result to True or False with Treshold function

In [193]:
y_pred=(y_pred>.5)# this means more than .5 returns True othewise returns False

In [194]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [196]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[1519   76]
 [ 207  198]]
0.8585


#### it is almost the same accuracy that we get in the train score in model that shows we got a vey good accuracy